In [2]:
import pandas as pd
import numpy as np

### Import data

In [2]:
# data_path = '../data/MDataFiles_Stage1/' #This is for stage 1
data_path = '../data/MDataFiles_Stage2/' #This is for stage 2


#Raw data
rankings = pd.read_csv(data_path + 'MMasseyOrdinals_thruDay128.csv') # name of this one changed in stage 2
detailed_reg_season = pd.read_csv(data_path +'MRegularSeasonDetailedResults.csv')
team_coaches = pd.read_csv(data_path + 'MTeamCoaches.csv')
detailed_tourney_results = pd.read_csv(data_path + 'MNCAATourneyDetailedResults.csv')
conf_tourney = pd.read_csv(data_path + 'MConferenceTourneyGames.csv')
team_conferences = pd.read_csv(data_path + 'MTeamConferences.csv')
tourney_seeds = pd.read_csv(data_path + 'MNCAATourneySeeds.csv')
teams = pd.read_csv(data_path + 'MTeams.csv')



In [7]:
regular_season_results = pd.read_csv('../data/MRegularSeasonDetailedResults.csv')
detailed_tourney_results = pd.read_csv('../data/MNCAATourneyDetailedResults.csv')


In [17]:
class PreProcess():

    win_cols = ['TourneyGame','Season', 'DayNum', 'WTeamID','LTeamID', 'WScore', 'LScore', 'WLoc', 
    'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
    'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
    'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']

    lose_cols = ['TourneyGame','Season', 'DayNum', 'LTeamID', 'WTeamID', 'LScore', 'WScore', 'WLoc',
    'NumOT', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3','LFTM', 'LFTA', 'LOR', 'LDR', 
    'LAst', 'LTO', 'LStl', 'LBlk', 'LPF','WFGM', 'WFGA', 'WFGM3', 'WFGA3', 
    'WFTM', 'WFTA', 'WOR', 'WDR','WAst', 'WTO', 'WStl', 'WBlk', 'WPF']

    new_cols = ['TourneyGame','Season', 'DayNum', 'Team1', 'Team2', 'Team1_score', 'Team2_score', 'WLoc',
    'NumOT', 'Team1_FGM', 'Team1_FGA', 'Team1_FGM3', 'Team1_FGA3', 'Team1_FTM', 'Team1_FTA', 'Team1_OR', 'Team1_DR',
    'Team1_Ast', 'Team1_TO', 'Team1_Stl', 'Team1_Blk', 'Team1_PF', 'Team2_FGM', 'Team2_FGA', 'Team2_FGM3', 'Team2_FGA3',
    'Team2_FTM', 'Team2_FTA', 'Team2_OR', 'Team2_DR', 'Team2_Ast', 'Team2_TO', 'Team2_Stl', 'Team2_Blk', 'Team2_PF']


    def __init__(self, tournament_data, regular_season_data):
        self.tournament_data = tournament_data
        self.regular_season_data = regular_season_data
    
    def process(self):


        self.tournament_data['TourneyGame'] = 1
        self.regular_season_data['TourneyGame'] = 0
        df = pd.concat([self.tournament_data, self.regular_season_data])
        
        #Remap column names
        df_winners = df[PreProcess.win_cols].rename(columns={i:j for (i,j) in zip(
                            PreProcess.win_cols, PreProcess.new_cols)}).copy()
        df_losers = df[PreProcess.lose_cols].rename(columns={i:j for (i,j) in zip(
                            PreProcess.lose_cols, PreProcess.new_cols)}).copy()
        
        #Add column to remember who won/lost
        df_winners['Outcome'] = 1
        df_losers['Outcome'] = 0
        
        #Encode location when Team1 is the winner
        df_winners['Loc'] = np.where(df_winners['WLoc'] == 'H', 'H', \
                                    np.where(df_winners['WLoc'] == 'A', 'A', 'N'))
        
        #Encode location when Team1 is the loser
        df_losers['Loc'] = np.where(df_losers['WLoc'] == 'H', 'A', \
                                    np.where(df_losers['WLoc'] == 'A', 'H', 'N'))
        
        #Create full df
        full = df_winners.append(df_losers)

        return full
            

    

In [18]:
processed_data = PreProcess(regular_season_results, detailed_tourney_results).process()

In [19]:
processed_data.head()

,TourneyGame,Season,DayNum,Team1,Team2,Team1_score,Team2_score,WLoc,NumOT,Team1_FGM,...,Team2_FTA,Team2_OR,Team2_DR,Team2_Ast,Team2_TO,Team2_Stl,Team2_Blk,Team2_PF,Outcome,Loc
0,1,2003,10,1104,1328,68,62,N,0,27,...,22,10,22,8,18,9,2,20,1,N
1,1,2003,10,1272,1393,70,63,N,0,26,...,20,20,25,7,12,8,6,16,1,N
2,1,2003,11,1266,1437,73,61,N,0,24,...,23,31,22,9,12,2,5,23,1,N
3,1,2003,11,1296,1457,56,50,N,0,18,...,15,17,20,9,19,4,3,23,1,N
4,1,2003,11,1400,1208,77,71,N,0,30,...,27,21,15,12,10,7,1,14,1,N
